In [0]:
!mkdir kaggle

In [0]:
import json
token = {"username":"piyushtiwari","key":"4f2d8ff0731a68d2096176ef5a75b22f"}
with open('/content/kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!mkdir ~/.kaggle

In [0]:
!cp /content/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets list -s korean

ref                                                              title                                              size  lastUpdated          downloadCount  
---------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
bryanpark/korean-single-speaker-speech-dataset                   Korean Single Speaker Speech Dataset                3GB  2019-11-15 22:18:08           1900  
usaf/korean-war-bombing-runs                                     Korean War Bombing Runs                           497KB  2017-09-13 19:22:29            235  
zldzmfoq12/korean-voice                                          korean_voice                                        5GB  2019-08-20 07:03:22             35  
gomjellie/kospi-price-data                                       Kospi Stock Price                                  42MB  2017-09-30 16:34:39            557  
rtatman/national-institute-of-the-korean-langu

In [0]:
!kaggle datasets download -d bryanpark/korean-single-speaker-speech-dataset

100% 2.86G/2.86G [00:47<00:00, 56.5MB/s]
100% 2.86G/2.86G [00:47<00:00, 64.2MB/s]


In [0]:
!pip install python_speech_features

In [0]:
import os

In [0]:
os.mkdir("speech")

In [0]:
!unzip "/content/{/content}/datasets/bryanpark/korean-single-speaker-speech-dataset/korean-single-speaker-speech-dataset.zip" -d "speech"

Archive:  /content/{/content}/datasets/bryanpark/korean-single-speaker-speech-dataset/korean-single-speaker-speech-dataset.zip
  inflating: speech/kss/1/1_0000.wav  
  inflating: speech/kss/1/1_0001.wav  
  inflating: speech/kss/1/1_0002.wav  
  inflating: speech/kss/1/1_0003.wav  
  inflating: speech/kss/1/1_0004.wav  
  inflating: speech/kss/1/1_0005.wav  
  inflating: speech/kss/1/1_0006.wav  
  inflating: speech/kss/1/1_0007.wav  
  inflating: speech/kss/1/1_0008.wav  
  inflating: speech/kss/1/1_0009.wav  
  inflating: speech/kss/1/1_0010.wav  
  inflating: speech/kss/1/1_0011.wav  
  inflating: speech/kss/1/1_0012.wav  
  inflating: speech/kss/1/1_0013.wav  
  inflating: speech/kss/1/1_0014.wav  
  inflating: speech/kss/1/1_0015.wav  
  inflating: speech/kss/1/1_0016.wav  
  inflating: speech/kss/1/1_0017.wav  
  inflating: speech/kss/1/1_0018.wav  
  inflating: speech/kss/1/1_0019.wav  
  inflating: speech/kss/1/1_0020.wav  
  inflating: speech/kss/1/1_0021.wav  
  inflating: sp

In [0]:
from sklearn.model_selection import train_test_split

# tensorflow imports
import tensorflow as tf
import keras

# tf.keras imports
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense, Conv1D, Activation, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization, Input, MaxPooling1D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model

# import for showing the confusion matrix

import numpy as np

In [0]:
from python_speech_features import mfcc
import os
import librosa

def wav_to_mfcc(wav_filename, num_cepstrum):
    """ extract MFCC features from a wav file
    
    :param wav_filename: filename with .wav format
    :param num_cepstrum: number of cepstrum to return
    :return: MFCC features for wav file
    """
    mfcc = []
    sig, rate = librosa.load(wav_filename)
    
    mfcc_features = librosa.feature.mfcc(sig, rate, n_mfcc=num_cepstrum)
    if len(mfcc_features[0])<152:
      mfcc_features = np.concatenate((mfcc_features, np.zeros((40, 152-len(mfcc_features[0])))), axis=1)
    else:
      mfcc_features = mfcc_features[:,:152]

    return mfcc_features

In [0]:
mfcc = wav_to_mfcc("/content/speech/kss/2/2_0003.wav", 40)

In [0]:
(mfcc[1][150])

0.0

In [0]:
len(os.listdir("/content/speech/kss/3"))

5025

In [0]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

In [0]:
mfcc= mfcc.transpose()
pt.fit(mfcc)
mfcc = pt.transform(mfcc)

In [0]:
mfcc[1][29]

-2.2568051601096086

In [0]:
def load_data(data_dir):
  label = 0
  labels = []
  data = []
  for folder in os.listdir(data_dir):
    count = 0
    each = os.path.join(data_dir, folder)
    
    data_each = []
    for speech in os.listdir(each):
      if count<1040:
        path = os.path.join(each, speech)
        mfcc = wav_to_mfcc(path, 40)
        mfcc= mfcc.transpose()
        pt.fit(mfcc)
        mfcc = pt.transform(mfcc)
        data_each.append(mfcc)
        labels.append(label)
        count+=1
    label+=1
    data.append(data_each)
  return data, labels

In [0]:
data_dir = "/content/speech/kss"
data, labels = load_data(data_dir)

In [0]:
data_arr = np.array(data)

In [0]:
data_arr.shape

(4, 1040, 152, 40)

In [0]:
data_arr = data_arr.reshape(-1,152,40)

In [0]:
data_arr.shape

(4160, 152, 40)

In [0]:
labels = tf.keras.utils.to_categorical(labels)

In [0]:
from sklearn.model_selection import train_test_split

X_train_big, X_test_big, Y_train_big, Y_test_big = train_test_split(data_arr, labels, test_size = 0.05, shuffle=True)

In [0]:
print(X_train.shape)

(910, 152, 40)


In [0]:
labels.shape

(4160, 4)

In [0]:
def create_model():
  model = Sequential()
  
  conv1 = model.add(Conv1D(80,kernel_size=3, padding='same', activation='relu', input_shape=(152, 40)))
  max1 = model.add(MaxPooling1D(pool_size=2))
  #drop1 = model.add(Dropout(0.3))
  bn1 = model.add(BatchNormalization())

  conv2 = model.add(Conv1D(128,kernel_size=3, padding='same', activation='relu'))
  max2 = model.add(MaxPooling1D(pool_size=2))
  #drop3 = model.add(Dropout(0.3))
  bn2 = model.add(BatchNormalization())

  conv3 = model.add(Conv1D(128,kernel_size=3, padding='same', activation='relu'))
  max3 = model.add(MaxPooling1D(pool_size=2))
  #drop3 = model.add(Dropout(0.3))
  bn3 = model.add(BatchNormalization())

  conv4 = model.add(Conv1D(256,kernel_size=3, padding='same', activation='relu'))
  max4 = model.add(MaxPooling1D(pool_size=2))
  #drop4 = model.add(Dropout(0.3))
  bn4 = model.add(BatchNormalization())

  conv5 = model.add(Conv1D(256,kernel_size=3, padding='same', activation='relu'))
  max5 = model.add(MaxPooling1D(pool_size=2))
  #drop5 = model.add(Dropout(0.3))
  bn5 = model.add(BatchNormalization())

  conv6 = model.add(Conv1D(512,kernel_size=3, padding='same', activation='relu'))
  max6 = model.add(MaxPooling1D(pool_size=2))
  bn6 = model.add(BatchNormalization())
  #drop6 = model.add(Dropout(0.5))

  conv7 = model.add(Conv1D(512,kernel_size=3, padding='same', activation='relu'))
  max7 = model.add(MaxPooling1D(pool_size=2))
  bn7 = model.add(BatchNormalization())
  #drop6 = model.add(Dropout(0.5))

  conv8 = model.add(Conv1D(1024,kernel_size=3, padding='same', activation='relu'))
  #max8 = model.add(MaxPooling1D(pool_size=2))
  bn8 = model.add(BatchNormalization())
  #drop6 = model.add(Dropout(0.5))
  
  fl = model.add(Flatten())
  #drop4 = model.add(Dropout(0.3))
  dense1 = model.add(Dense(2048, activation='relu', kernel_regularizer = regularizers.l2(0.01)))

  dense2 = model.add(Dense(4, activation='softmax'))
  
  sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
  
  model.compile(optimizer = sgd, loss = tf.keras.losses.categorical_crossentropy, metrics = ["accuracy"])   
  model.summary()
  
  return model

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2, min_lr=0.0001)
model = create_model()
model_hist = model.fit(X_train_big, Y_train_big, epochs = 80, batch_size = 16, validation_split=0.05, callbacks=[reduce_lr]) 

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_130 (Conv1D)          (None, 152, 80)           9680      
_________________________________________________________________
max_pooling1d_118 (MaxPoolin (None, 76, 80)            0         
_________________________________________________________________
batch_normalization_129 (Bat (None, 76, 80)            320       
_________________________________________________________________
conv1d_131 (Conv1D)          (None, 76, 128)           30848     
_________________________________________________________________
max_pooling1d_119 (MaxPoolin (None, 38, 128)           0         
_________________________________________________________________
batch_normalization_130 (Bat (None, 38, 128)           512       
_________________________________________________________________
conv1d_132 (Conv1D)          (None, 38, 128)         

In [0]:
model.save('my_model_new_2.h5')

In [0]:
model.evaluate(X_test_big, Y_test_big, verbose=2)

208/208 - 0s - loss: 1.1088 - acc: 0.7212


[1.108823299407959, 0.72115386]

In [0]:
!wget "https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip"

--2019-12-24 10:06:22--  https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225505317 (215M) [application/octet-stream]
Saving to: ‘Audio_Song_Actors_01-24.zip’

Audio_Song_Actors_0 100%[===================>] 215.06M  89.2MB/s    in 2.4s    

2019-12-24 10:06:25 (89.2 MB/s) - ‘Audio_Song_Actors_01-24.zip’ saved [225505317/225505317]



In [0]:
os.mkdir("speech_emotion")

In [0]:
!unzip \*.zip -d "speech_emotion"

Archive:  Audio_Song_Actors_01-24.zip
   creating: speech_emotion/Actor_01/
  inflating: speech_emotion/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-01-01-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-01-01-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-01-02-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-01-02-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-02-01-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-02-01-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-02-02-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-02-02-02-02-01.wav  
  inflating: speech_emotion/Actor_01/03-02-03-01-01-01-01.wav  
  inflating: speech_emotion/Actor_01/03-02-03-01-01-02-01.wav  
  inflating: speech_emotion/

In [0]:
for i in os.listdir("/content/speech_emotion"):
  path = os.path.join("/content/speech_emotion",i)
  print(len(os.listdir(path)))

In [0]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

In [0]:
def load_data(data_dir):
  data = []
  labels = []

  for folder in os.listdir(data_dir):
    path_ = os.path.join(data_dir,folder)
    data_each = []

    for i in os.listdir(path_):
      path_final = os.path.join(path_,i)
      
      mfcc = wav_to_mfcc(path_final, 40)
      mfcc= mfcc.transpose()
      pt.fit(mfcc)
      mfcc = pt.transform(mfcc)
      data_each.append(mfcc)
           
      str = i[6:8]
      if(str == "01"):
        labels.append(1)
      elif(str == "02"):
        labels.append(2)
      elif(str == "03"):
        labels.append(3)
      elif(str == "04"):
        labels.append(4)
      elif(str == "05"):
        labels.append(5)
      elif(str == "06"):
        labels.append(6)
      elif(str == "07"):
        labels.append(7)
      elif(str == "08"):
        labels.append(8)

      labels = labels
  
    data.append(data_each)    

  labels = tf.keras.utils.to_categorical(labels)
  return data,labels

In [0]:
data_dir = "/content/speech_emotion"
data_emotion,labels_emotion = load_data(data_dir)

In [0]:
data_emotion_arr = np.array(data_emotion_arr)

In [0]:
data_emotion_arr.shape

(23, 44, 152, 40)

In [0]:
data_emotion_arr = data_emotion_arr.reshape(-1, 152, 40)

In [0]:
del(data_emotion_list[11])

In [0]:
model = load_model('my_model_new.h5')

In [0]:
for layers in model.layers:
  print(layers)

In [0]:
model = load_model('my_model_new.h5')

new_model = Model(model.inputs, model.layers[-11].output)
for layer in new_model.layers:
  layer.trainable = False

new_model.set_weights(model.get_weights()[:32])

model_emotion = Sequential()
model_emotion.add(new_model)

model_emotion.add(Conv1D(64,kernel_size=3, padding='same', activation='relu'))
model_emotion.add(MaxPooling1D(pool_size=2))
model_emotion.add(BatchNormalization())
model_emotion.add(Dropout(0.5))

model_emotion.add(Flatten())
model_emotion.add(Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.01)))
model_emotion.add(Dense(7, activation='softmax'))

sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model_emotion.compile(optimizer = sgd, loss = tf.keras.losses.categorical_crossentropy, metrics = ["accuracy"])   
model_emotion.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_13 (Model)             (None, 4, 512)            782352    
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 4, 64)             98368     
_________________________________________________________________
max_pooling1d_63 (MaxPooling (None, 2, 64)             0         
_________________________________________________________________
batch_normalization_64 (Batc (None, 2, 64)             256       
_________________________________________________________________
dropout_19 (Dropout)         (None, 2, 64)             0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)              

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data_emotion_arr, labels_emotion, test_size = 0.1, shuffle=True)

In [0]:
X_train.shape

(910, 152, 40)

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2,
                              patience=2, min_lr=0.0001)
model_hist = model_emotion.fit(X_train, Y_train, epochs = 50, batch_size = 16, validation_split=0.05, callbacks=[reduce_lr]) 

Train on 864 samples, validate on 46 samples
Epoch 1/50
864/864 [==============================] - 6s 7ms/sample - loss: 2.8827 - acc: 0.2350 - val_loss: 2.5900 - val_acc: 0.3043
Epoch 2/50
864/864 [==============================] - 0s 480us/sample - loss: 2.3508 - acc: 0.3924 - val_loss: 2.3963 - val_acc: 0.4130
Epoch 3/50
864/864 [==============================] - 0s 505us/sample - loss: 2.1184 - acc: 0.4838 - val_loss: 2.2740 - val_acc: 0.3696
Epoch 4/50
864/864 [==============================] - 0s 474us/sample - loss: 1.9454 - acc: 0.5590 - val_loss: 2.2223 - val_acc: 0.3913
Epoch 5/50
864/864 [==============================] - 0s 477us/sample - loss: 1.7882 - acc: 0.6053 - val_loss: 2.0703 - val_acc: 0.5435
Epoch 6/50
864/864 [==============================] - 0s 530us/sample - loss: 1.6824 - acc: 0.6516 - val_loss: 1.9569 - val_acc: 0.5217
Epoch 7/50
864/864 [==============================] - 0s 481us/sample - loss: 1.5694 - acc: 0.6852 - val_loss: 2.0037 - val_acc: 0.5435
Epoch

In [0]:
model_emotion.evaluate(X_test, Y_test, verbose=2)

102/102 - 0s - loss: 1.8070 - acc: 0.6176


[1.8069733825384402, 0.61764705]

In [0]:
model_emotion.save("model_emotion_61%")